In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tqdm import tqdm

import numpy as np
import pandas as pd

import re

import matplotlib.pyplot as plt

import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
# from torch.nn.functional import relu, sigmoid
from torch import optim

import torch
from torch.utils.data import Dataset

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/captions.txt')

In [12]:
df.head(10)

,image,caption
0,1000268201_693b08cb0e.jpg,child in pink dress is climbing up set of stai...
1,1000268201_693b08cb0e.jpg,girl going into wooden building
2,1000268201_693b08cb0e.jpg,little girl climbing into wooden playhouse
3,1000268201_693b08cb0e.jpg,little girl climbing the stairs to her playhouse
4,1000268201_693b08cb0e.jpg,little girl in pink dress going into wooden cabin
5,1001773457_577c3a7d70.jpg,black dog and spotted dog are fighting
6,1001773457_577c3a7d70.jpg,black dog and tricolored dog playing with each...
7,1001773457_577c3a7d70.jpg,black dog and white dog with brown spots are s...
8,1001773457_577c3a7d70.jpg,two dogs of different breeds looking at each o...
9,1001773457_577c3a7d70.jpg,two dogs on pavement moving toward each other


In [25]:
df1=df[df['image'] == '1000268201_693b08cb0e.jpg']
df1.head(10)

,image,caption
0,1000268201_693b08cb0e.jpg,child in pink dress is climbing up set of stai...
1,1000268201_693b08cb0e.jpg,girl going into wooden building
2,1000268201_693b08cb0e.jpg,little girl climbing into wooden playhouse
3,1000268201_693b08cb0e.jpg,little girl climbing the stairs to her playhouse
4,1000268201_693b08cb0e.jpg,little girl in pink dress going into wooden cabin


In [5]:

def clean_caption(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join([w for w in text.split() if len(w) > 1])
    return text

df['caption'] = df['caption'].apply(clean_caption)


In [26]:
captions = df["caption"].tolist()
max_length = max(len(caption.split()) for caption in captions)
max_length

32

In [ ]:
# captions

In [6]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


In [27]:
features1 = pd.read_csv('/content/drive/MyDrive/final1_feature.csv')

In [28]:
features1.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919
0,1000268201_693b08cb0e.jpg,0.000079,0.000735,0.001140,0.003629,0.000718,0.000175,0.000589,0.000237,0.000014,...,0.140825,2.668942,0.882584,0.732694,0.289506,0.033662,0.154522,0.565233,0.229038,0.696399
1,1001773457_577c3a7d70.jpg,0.000125,0.000452,0.001929,0.002451,0.000430,0.000105,0.000530,0.000363,0.000113,...,2.349692,0.034021,0.248741,2.027267,0.373612,2.619257,1.265228,0.074858,2.047389,0.090313
2,1002674143_1b742ab4b8.jpg,0.000050,0.000371,0.001941,0.001252,0.000919,0.000310,0.000894,0.000446,0.000085,...,0.122315,0.363583,0.243627,1.168989,1.221152,0.578688,0.222599,0.087597,0.468917,0.317462
3,1003163366_44323f5815.jpg,0.000141,0.000779,0.001270,0.005102,0.000638,0.000237,0.000250,0.000151,0.000090,...,0.158818,2.555785,0.104294,0.079707,0.401555,1.361094,0.714018,0.223578,0.081238,0.133313
4,1007129816_e794419615.jpg,0.000098,0.000725,0.001890,0.001837,0.000844,0.000168,0.000611,0.000541,0.000104,...,0.002541,0.070152,0.083582,1.529452,0.656722,0.482287,0.973176,1.927115,1.369830,0.171644


In [29]:
features = pd.read_csv('/content/drive/MyDrive/features.csv')

In [30]:
features.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1000268201_693b08cb0e.jpg,0.000530,0.020955,0.000000,0.000478,0.102640,2.816984,0.000314,0.000894,0.500193,...,5.755997,0.000000,11.968189,0.855762,2.582031,0.000000,2.545345,3.693776,3.893597,5.397244
1,1001773457_577c3a7d70.jpg,0.000896,0.004986,0.001943,0.005952,0.012798,0.276479,0.001835,0.012086,0.115777,...,0.703914,5.111288,5.492939,10.403438,0.000000,0.000000,1.604755,3.473159,0.753371,13.305740
2,1002674143_1b742ab4b8.jpg,0.000139,0.010884,0.000000,0.001002,0.310223,0.572058,0.001189,0.001960,0.230935,...,2.796671,0.000000,2.526081,9.665098,3.163004,0.000000,4.403409,0.000000,2.451269,15.814799
3,1003163366_44323f5815.jpg,0.000156,0.017315,0.001471,0.002552,0.038932,1.582965,0.001269,0.003844,0.227926,...,1.709077,2.349216,6.488191,12.943340,0.000000,0.000000,3.434384,2.654669,10.393051,2.786068
4,1007129816_e794419615.jpg,0.000144,0.001897,0.008756,0.018821,0.000000,0.000000,0.002928,0.002050,0.459453,...,2.115044,4.440852,8.961600,5.716276,2.507264,2.447746,0.538589,0.000000,7.990150,2.681692


In [38]:
features.shape, features1.shape, type(features)

((1024,), (8091, 1921), numpy.ndarray)

In [32]:
result_dict = features.to_dict(orient='index')
output_dict = {}
for key, value in result_dict.items():
    image_name = value['Unnamed: 0']
    features = np.array([value[str(i)] for i in range(len(value) - 1)])
    output_dict[image_name] = features.reshape(1, -1)
# print(output_dict)

In [40]:
# output_dict.shape
type(output_dict)

dict

In [47]:
# output_dict.keys()
# output_dict['1000268201_693b08cb0e.jpg'].shape
len(output_dict['1000268201_693b08cb0e.jpg'])


1

In [33]:
sample_feat = next(iter(output_dict.values()))  # (1, F)
IMG_FEAT_DIM = sample_feat.shape[-1]
print("Image‑feature dim =", IMG_FEAT_DIM)

Image‑feature dim = 1024


In [50]:
sample_feat

array([[5.3009967e-04, 2.0954693e-02, 0.0000000e+00, ..., 3.6937761e+00,
        3.8935966e+00, 5.3972435e+00]])

In [37]:
# type(IMG_FEAT_DIM), IMG_FEAT_DIM
sample_feat.shape , type(sample_feat)

((1, 1024), numpy.ndarray)

In [ ]:
from torch.utils.data import Dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import numpy as np

class BERTCaptionDataset(Dataset):
    def __init__(self, df, image_features, tokenizer, max_length=30):
        self.df = df.reset_index(drop=True)
        self.image_features = image_features
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["image"]
        caption = row["caption"]

        # ویژگی‌های تصویر
        img_feat = self.image_features[image_id]
        if isinstance(img_feat, np.ndarray) and len(img_feat.shape) == 2 and img_feat.shape[0] == 1:
            img_feat = img_feat.squeeze(0)
        img_feat = torch.tensor(img_feat, dtype=torch.float32)

        # توکنایز کپشن
        tokens = self.tokenizer.encode(caption, add_special_tokens=False)
        input_tokens = tokens[:-1]
        target_tokens = tokens[1:]

        input_tokens_padded = pad_sequences([input_tokens], maxlen=self.max_length, padding='post')[0]
        target_tokens_padded = pad_sequences([target_tokens], maxlen=self.max_length, padding='post')[0]

        input_tensor = torch.tensor(input_tokens_padded, dtype=torch.long)
        target_tensor = torch.tensor(target_tokens_padded, dtype=torch.long)

        return (img_feat, input_tensor), target_tensor


# **test**

In [52]:
from torch.utils.data import Dataset
import torch
import numpy as np

class BERTCaptionDataset(Dataset):
    def __init__(
        self,
        df,
        image_features: dict,
        tokenizer,
        max_length: int = 30
    ):
        self.df = df.reset_index(drop=True)
        self.image_features = image_features
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["image"]
        caption = row["caption"]

        # -------- Image features --------
        img_feat = self.image_features[image_id]
        if isinstance(img_feat, np.ndarray):
            img_feat = torch.from_numpy(img_feat).float()
        else:
            img_feat = torch.tensor(img_feat, dtype=torch.float32)

        if img_feat.ndim == 2 and img_feat.size(0) == 1:
            img_feat = img_feat.squeeze(0)

        # -------- Tokenization (BERT style) --------
        encoding = self.tokenizer(
            caption,
            add_special_tokens=True,            # [CLS] ... [SEP]
            max_length=self.max_length + 1,     # چون shift می‌کنیم
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        # -------- Shift for decoder --------
        input_tokens = input_ids[:-1]
        target_tokens = input_ids[1:]
        attention_mask = attention_mask[:-1]

        return {
            "image_features": img_feat,
            "input_ids": input_tokens,
            "attention_mask": attention_mask,
            "labels": target_tokens
        }


# **this one worked thebest way**
Problems/Weaknesses of the Current Version

❌ Use of
This is formatted as code
```
#tensorflow.keras.preprocessing.sequence.pad_sequences
```

In PyTorch project (unnecessary and slow)

❌ No use of BERT specific tokens
[CLS], [SEP], [PAD]

❌ No return of attention_mask

❌ No control over actual BERT truncation

❌ Dependency on encode instead of new HuggingFace API

Modern and clean version (recommended 🔥)
✅ Compatible with BERT tokenizer (HuggingFace)
✅ Pure PyTorch
✅ Ready for Decoder sequence generation

In [51]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [53]:
image_ids = list(output_dict.keys())
# captions_with_ids = list(zip(image_ids, captions))
dataset = BERTCaptionDataset(df,output_dict, tokenizer)


In [54]:
from torch.utils.data import DataLoader

# train_dataset = BERTCaptionDataset(df, output_dict, tokenizer)
train_loader  = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
for (X1, X2), y in train_loader:
    print("🔹 Image feature batch shape:", X1.shape)  # [batch_size, feature_dim]
    print("🔹 Attention mask batch shape:", X2.shape) # [batch_size, seq_len]
    print("🔹 Input ids batch shape:", y.shape)       # [batch_size, seq_len]
    break

🔹 Image feature batch shape: torch.Size([8, 1024])
🔹 Attention mask batch shape: torch.Size([8, 30])
🔹 Input ids batch shape: torch.Size([8, 30])


# **test**

In [57]:
for batch in train_loader:
    X_img = batch["image_features"]
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    print("🔹 Image feature batch shape:", X_img.shape)       # [B, feat_dim]
    print("🔹 Input ids batch shape:", input_ids.shape)      # [B, seq_len]
    print("🔹 Attention mask batch shape:", attention_mask.shape)  # [B, seq_len]
    print("🔹 Labels batch shape:", labels.shape)            # [B, seq_len]

    break


🔹 Image feature batch shape: torch.Size([8, 1024])
🔹 Input ids batch shape: torch.Size([8, 30])
🔹 Attention mask batch shape: torch.Size([8, 30])
🔹 Labels batch shape: torch.Size([8, 30])


# **test**

In [58]:
import torch
import torch.nn as nn
from transformers import BertModel


class BERTUniLSTMResidual(nn.Module):
    def __init__(
        self,
        img_feat_dim,
        hidden_dim,
        vocab_size,
        dropout_rate=0.3,
        freeze_bert=True
    ):
        super().__init__()

        # -------- BERT --------
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.embedding = self.bert.embeddings.word_embeddings
        embed_dim = self.embedding.embedding_dim  # 768

        if freeze_bert:
            for p in self.embedding.parameters():
                p.requires_grad = False

        # -------- Image projection --------
        self.img_fc = nn.Linear(img_feat_dim, embed_dim)

        # -------- LSTM --------
        self.lstm = nn.LSTM(
            input_size=embed_dim * 2,
            hidden_size=hidden_dim,
            batch_first=True
        )

        # -------- Residual projection --------
        self.res_proj = nn.Linear(embed_dim * 2, hidden_dim, bias=False)

        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, img_feat, input_ids, attention_mask=None):
        """
        img_feat: (B, img_feat_dim)
        input_ids: (B, L)
        attention_mask: (B, L)  [optional]
        """

        # -------- Embeddings --------
        txt_emb = self.embedding(input_ids)          # (B, L, E)
        img_emb = self.img_fc(img_feat)              # (B, E)

        seq_len = input_ids.size(1)
        img_emb = img_emb.unsqueeze(1).expand(-1, seq_len, -1)

        combined = torch.cat([txt_emb, img_emb], dim=-1)  # (B, L, 2E)

        # -------- LSTM --------
        lstm_out, _ = self.lstm(combined)             # (B, L, H)

        # -------- Residual --------
        out = lstm_out + self.res_proj(combined)

        out = self.dropout(out)
        logits = self.fc(out)                         # (B, L, vocab)

        return logits


# **2 test**

In [59]:
vocab_size = tokenizer.vocab_size

model = BERTUniLSTMResidual(
    img_feat_dim=IMG_FEAT_DIM,
    hidden_dim=512,
    vocab_size=vocab_size,
    dropout_rate=0.3,
    freeze_bert=True
)

model = model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [60]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-2
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    patience=2,
    factor=0.3
)

criterion = nn.CrossEntropyLoss(
    ignore_index=tokenizer.pad_token_id
)

num_epochs = 10
clip_value = 5.0

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):

        img_feat = batch["image_features"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        logits = model(
            img_feat=img_feat,
            input_ids=input_ids,
            attention_mask=attention_mask
        )   # (B, L, vocab)

        loss = criterion(
            logits.view(-1, logits.size(-1)),
            labels.view(-1)
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: loss = {avg_loss:.4f}")

    scheduler.step(avg_loss)


Epoch 1/10: 100%|██████████| 5057/5057 [02:12<00:00, 38.19it/s]


Epoch 1: loss = 4.7328


Epoch 2/10: 100%|██████████| 5057/5057 [02:10<00:00, 38.68it/s]


Epoch 2: loss = 3.9703


Epoch 3/10: 100%|██████████| 5057/5057 [02:12<00:00, 38.14it/s]


...

Epoch 8: loss = 3.1807


Epoch 9/10: 100%|██████████| 5057/5057 [02:05<00:00, 40.37it/s]


Epoch 9: loss = 3.1284


Epoch 10/10: 100%|██████████| 5057/5057 [02:09<00:00, 38.95it/s]

Epoch 10: loss = 3.0849


# **almost took me around 21 minuts**

# **test**

In [61]:
import torch
import torch.nn.functional as F


def generate_caption(
    model,
    tokenizer,
    image_feature,
    device="cpu",
    max_length=30,
    top_k=1,
    temperature=1.0
):
    """
    خروجی: string کپشن (بدون CLS / SEP)
    """

    model.eval()

    # -------- image feature --------
    if isinstance(image_feature, torch.Tensor):
        img = image_feature.detach()
    else:
        img = torch.tensor(image_feature, dtype=torch.float32)

    if img.dim() == 1:
        img = img.unsqueeze(0)
    img = img.to(device)

    # -------- start token --------
    cls_id = tokenizer.cls_token_id
    sep_id = tokenizer.sep_token_id
    pad_id = tokenizer.pad_token_id

    seq_ids = [cls_id]

    with torch.no_grad():
        for _ in range(max_length):

            seq_tensor = torch.tensor(
                [seq_ids],
                dtype=torch.long,
                device=device
            )

            attention_mask = (seq_tensor != pad_id).long()

            logits = model(
                img_feat=img,
                input_ids=seq_tensor,
                attention_mask=attention_mask
            )

            next_logits = logits[:, -1, :] / temperature

            # -------- greedy --------
            if top_k == 1:
                next_id = torch.argmax(next_logits, dim=-1).item()

            # -------- top-k sampling --------
            else:
                vals, idxs = torch.topk(next_logits, k=top_k, dim=-1)
                probs = F.softmax(vals, dim=-1)
                next_id = idxs[0, torch.multinomial(probs, 1)].item()

            # -------- stop --------
            if next_id == sep_id:
                break

            # جلوگیری از تولید PAD
            if next_id == pad_id:
                continue

            seq_ids.append(next_id)

    caption = tokenizer.decode(
        seq_ids[1:],      # حذف CLS
        skip_special_tokens=True
    )

    return caption


In [62]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


the small dog is running through the air on grass with his mouth wide


In [63]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


the small white dog is jumping through the grass


In [64]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


small dog is playing on grass


In [65]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


the white dog is jumping up on the grass


In [66]:
test_feat = output_dict["103106960_e8a41d64f8.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


little boy in red jacket is kneeling on the sidewalk


In [67]:
test_feat = output_dict["131632409_4de0d4e710.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


little child is standing on beach


In [69]:
test_feat = output_dict["12830823_87d2654e31.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


children playing in the pool while another man is watching the side of the side of building


In [70]:
test_feat = output_dict["10815824_2997e03d76.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


two dogs are playing in grassy area


In [71]:
test_feat = output_dict["49553964_cee950f3ba.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


boy in red shirt and red life jacket is jumping over the lake


In [72]:
test_feat = output_dict["113678030_87a6a6e42e.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)

man is standing on ski lift


In [73]:
test_feat = output_dict["166654939_80ea4ddbcc.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


two women are racing cars racing on the track


In [74]:
test_feat = output_dict["221973402_ecb1cd51f1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


the little girl in the red shirt is running on the grass with red ball
